## langgraph - 持久化与记忆
****
- 基本运用：线程隔离的持久化层
- 基本运用：跨线程持久化调用
- 记忆：短期记忆的实现
- 记忆：长期以及实现
- 记忆：使用总结技术优化记忆


### 线程隔离的持久化层
*****

In [ ]:
from langchain_deepseek import ChatDeepSeek
import os
from langgraph.graph import StateGraph, MessagesState, START

model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)


def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
graph = builder.compile()

没有激活持久化层，无法实现多轮对话

In [3]:
input_message = {"role": "user", "content": "hi! 我是tomie"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

input_message = {"role": "user", "content": "我叫什么名字?"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! 我是tomie
================================== Ai Message ==================================

你好，Tomie！😊 很高兴认识你～有什么我可以帮你的吗？或者只是想打个招呼？✨
================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

你还没有告诉我你的名字呢！😊 你可以告诉我你的名字，我会记住的～或者，如果你是在问我的名字，我是 **DeepSeek Chat**，你可以叫我 **小深** 或 **DeepSeek**！很高兴认识你！✨


激活持久性层

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
# 使用 MemorySaver 保存中间状态
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [5]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "hi! 我是tomie"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! 我是tomie
================================== Ai Message ==================================

你好，Tomie！😊 很高兴认识你～有什么我可以帮你的吗？或者只是想打个招呼？


In [6]:
input_message = {"role": "user", "content": "我叫什么名字?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

哈哈，你刚刚说过啦！你叫 **Tomie**～ 😄  
（如果记错了，随时提醒我哦！需要我记住这个名字吗？）


注意thread_id的输入

In [7]:
input_message = {"role": "user", "content": "我叫什么名字?"}
for chunk in graph.stream(
    {"messages": [input_message]},
    {"configurable": {"thread_id": "2"}}, # different thread_id
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

目前我无法直接知道你的名字，但如果你愿意告诉我，我会记住并在接下来的对话中使用哦！😊 或者，你可以叫我“小助手”或任何你喜欢的名字～


### 跨线程共享持久化数据
****
- userid

设置内存记忆

In [ ]:
from langgraph.store.memory import InMemoryStore
from langchain_openai import OpenAIEmbeddings
import os
# 使用OpenAI的封装，但是运行国产嵌入模型
# 使用内存存储来保存向量化后记忆数据
in_memory_store = InMemoryStore(
    index={
        "embed": OpenAIEmbeddings(
            model="Pro/BAAI/bge-m3",
            api_key=os.environ.get("DEEPSEEK_API_KEY"),
            base_url=os.environ.get("DEEPSEEK_API_BASE")+ "/v1",
            ),
        "dims": 1024,
    }
)

In [8]:
import uuid
from typing import Annotated
from typing_extensions import TypedDict

from langchain_deepseek import ChatDeepSeek
import os
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore


model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)


# 注意：我们将 Store 参数传递给节点 --
# 这是我们编译图时使用的 Store
def call_model(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    # 从存储中检索用户信息
    user_id = config["configurable"]["user_id"]
    # 从存储中检索用户信息
    namespace = ("memories", user_id)
    memories = store.search(namespace, query=str(state["messages"][-1].content))
    info = "\n".join([d.value["data"] for d in memories])
    system_msg = f"你是一个正在与用户交谈的小助手。用户信息：{info}"

    # 如果用户要求模型记住信息，则存储新的记忆
    last_message = state["messages"][-1]
    if "记住" in last_message.content.lower() or "remember" in last_message.content.lower():
        # 硬编码一个记忆
        memory = "用户名字是tomiezhang"
        store.put(namespace, str(uuid.uuid4()), {"data": memory})

    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")

# 注意：我们在编译图时传递了 store 对象
graph = builder.compile(checkpointer=MemorySaver(), store=in_memory_store)

注意线程ID和用户ID

In [9]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
input_message = {"role": "user", "content": "请记住我的名字叫tomiezhang!"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

请记住我的名字叫tomiezhang!
================================== Ai Message ==================================

好的，Tomiezhang！我已经记住你的名字啦～ 以后会这样称呼你，有什么需要随时告诉我哦！(๑•̀ㅂ•́)و✧


跨线程使用相同的用户ID查询

In [10]:
# 注意线程ID和用户ID
config = {"configurable": {"thread_id": "2", "user_id": "1"}}
input_message = {"role": "user", "content": "我叫什么名字?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

你的名字是Tomie Zhang。需要我帮你记住其他信息吗? 😊


我们可以查询存储在内存中的记忆

In [11]:
for memory in in_memory_store.search(("memories", "1")):
    print(memory.value)

{'data': '用户名字是Tomie'}
{'data': '用户名字是tomiezhang'}


用户级的记忆隔离

In [12]:
config = {"configurable": {"thread_id": "3", "user_id": "2"}}
input_message = {"role": "user", "content": "我叫什么?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

我叫什么?
================================== Ai Message ==================================

目前我无法直接知道您的名字，但如果您愿意告诉我，我会记住并在对话中使用它！您叫什么名字呢？ 😊


### 短期记忆的实现
*****
- 基于最简单的ReAct智能体

In [ ]:
from typing import Literal

from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode

# 注意：使用内存存储来存储记忆
memory = MemorySaver()


@tool
def search(query: str):
    """调用此函数可以浏览网络。"""
    # 模拟一个网络搜索返回
    return "北京天气晴朗 大约22度 湿度30%"


tools = [search]
tool_node = ToolNode(tools)
model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState):
    """返回下一个要执行的节点。"""
    last_message = state["messages"][-1]
    # 如果没有函数调用，则结束
    if not last_message.tool_calls:
        return END
    # 否则如果有，我们继续
    return "action"


# 定义调用模型的函数
def call_model(state: MessagesState):
    response = bound_model.invoke(state["messages"])
    # 我们返回一个列表，因为这会被添加到现有列表中
    return {"messages": response}


# 定义一个图
workflow = StateGraph(MessagesState)

# 定义我们将在其间循环的两个节点
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# 将入口点设置为 `agent`
# 这意味着这个节点是第一个被调用的
workflow.add_edge(START, "agent")

# 现在我们添加一个条件边
workflow.add_conditional_edges(
    # 首先，我们定义起始节点。我们使用 `agent`。
    # 这意味着这些是在 `agent` 节点被调用后采取的边。
    "agent",
    # 接下来，我们传入将确定下一个调用哪个节点的函数。
    should_continue,
    # 接下来，我们传入路径映射 - 这条边可能去往的所有可能节点
    ["action", END],
)

# 现在我们从 `tools` 到 `agent` 添加一个普通边。
# 这意味着在调用 `tools` 之后，接下来调用 `agent` 节点。
workflow.add_edge("action", "agent")

# 最后，我们编译它！
# 这将它编译成一个 LangChain Runnable，
# 意味着你可以像使用任何其他 runnable 一样使用它
# 设置检查点为内存形式，注意没有设置store
app = workflow.compile(checkpointer=memory)


调用图

In [15]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! 我是tomie")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


input_message = HumanMessage(content="我叫什么名字?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! 我是tomie
================================== Ai Message ==================================

你好，Tomie！很高兴认识你！有什么可以帮你的吗？ 😊
================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

你刚刚告诉我你的名字是 **Tomie**！😊 有什么我可以帮你的吗？


### 长期记忆
****
- 使用MongDB

- 在本地安装mongodb
- windows:

In [ ]:
# 安装后，启动MongoDB服务
# 通常安装后会自动设置为系统服务并启动
# 如果没有自动启动，可以在命令提示符中运行：
net start MongoDB

- macos

In [ ]:
# 安装
brew tap mongodb/brew
brew install mongodb-community

# 启动服务
brew services start mongodb-community


- 使用docker快速安装

In [ ]:
# 拉取MongoDB镜像
docker pull mongo

# 运行MongoDB容器
docker run -d -p 27017:27017 --name mongodb mongo

# 验证容器是否运行
docker ps


连接mongodb

In [21]:
! pip install -U pymongo langgraph langgraph-checkpoint-mongodb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 471.0 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pymongo
    Found existing installation: pymongo 4.11.2
    Uninstalling pymongo-4.11.2:
      Successfully uninstalled pymongo-4.11.2
  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 2.0.18
    Uninstalling langgraph-checkpoint-2.0.18:
      Successfully uninstalled langgraph-checkpoint-2.0.18

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


测试MongoDB连接

In [22]:
import pymongo

# 创建MongoDB客户端连接
client = pymongo.MongoClient("mongodb://localhost:27017/")

# 测试连接
try:
    client.admin.command('ping')
    print("MongoDB连接成功！")
except Exception as e:
    print(f"MongoDB连接失败: {e}")


MongoDB连接成功！


创建一个最简单的智能体

In [23]:
from typing import Literal

from langchain_core.tools import tool
from langchain_deepseek import ChatDeepSeek
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["北京", "深圳"]):
    """用来返回天气信息的工具函数。"""
    if city == "北京":
        return "北京天气晴朗 大约22度 湿度30%"
    elif city == "深圳":
        return "深圳天气多云 大约28度 湿度80%"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)

连接mongodb进行查询

In [24]:
from langgraph.checkpoint.mongodb import MongoDBSaver

MONGODB_URI = "localhost:27017"  # replace this with your connection string

with MongoDBSaver.from_conn_string(MONGODB_URI) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    response = graph.invoke(
        {"messages": [("human", "北京今天的天气如何？")]}, config
    )

In [25]:
print(response)

{'messages': [HumanMessage(content='北京今天的天气如何？', additional_kwargs={}, response_metadata={}, id='6c251bbb-1cd6-4d59-992e-271ace17d487'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0195e7286817736e97482bc1cbf33a60', 'function': {'arguments': '{"city":"北京"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 90, 'total_tokens': 109, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-731e5cde-1014-4ef7-9fb8-7fe015b03a98-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '北京'}, 'id': '0195e7286817736e97482bc1cbf33a60', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 19, 'total_tokens': 109, 'input_token_details': {}, 'output_token_details': {}}), ToolMessage(content='北京天气晴朗 大约22度 湿度30%', name='get_weathe

### 优化记忆
****
- 消息过滤：对旧消息进行类似删除或编辑的操作，目的是为了防止撑爆上下文
- 消息总结：对旧消息进行总结，目的一样是为了防止记忆内容过长
- 注意对记忆的管理是一项关于召回率和精度的平衡艺术

In [26]:
from typing import Literal

from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode

memory = MemorySaver()


@tool
def search(query: str):
    """调用此函数可以浏览网络。"""
    # 模拟一个网络搜索返回
    return "北京天气晴朗 大约22度 湿度30%"


tools = [search]
tool_node = ToolNode(tools)
model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState):
    """返回下一个要执行的节点。"""
    last_message = state["messages"][-1]
    # 如果没有函数调用，则结束
    if not last_message.tool_calls:
        return END
    # 否则，如果有函数调用，我们继续
    return "action"


def filter_messages(messages: list):
    # 这是一个非常简单的辅助函数，它只使用最后一条消息
    return messages[-1:]


# 定义调用模型的函数
def call_model(state: MessagesState):
    messages = filter_messages(state["messages"])
    response = bound_model.invoke(messages)
    # 我们返回一个列表，因为这将被添加到现有列表中
    return {"messages": response}


# 定义一个新图
workflow = StateGraph(MessagesState)

# 定义我们将在其间循环的两个节点
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# 将入口点设置为 `agent`
# 这意味着这个节点是第一个被调用的
workflow.add_edge(START, "agent")

# 现在添加一个条件边
workflow.add_conditional_edges(
    # 首先，我们定义起始节点。我们使用 `agent`。
    # 这意味着这些是在调用 `agent` 节点后采取的边。
    "agent",
    # 接下来，我们传入将确定下一个调用哪个节点的函数。
    should_continue,
    # 接下来，我们传入路径图 - 此边可能去往的所有可能节点
    ["action", END],
)

# 现在我们从 `action` 到 `agent` 添加一个普通边。
# 这意味着在调用 `action` 之后，下一步调用 `agent` 节点。
workflow.add_edge("action", "agent")

# 最后，我们编译它！
# 这将它编译成一个 LangChain Runnable，
# 意味着你可以像使用任何其他 runnable 一样使用它
app = workflow.compile(checkpointer=memory)

进行调用，由于进行了消息裁剪，所以即便使用了memory，依然无法记住过往对话

In [27]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! 我是tomie")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 请注意，我们在这里使用了一个辅助函数，它只使用最后一条消息
# 这将导致我们的模型只看到最后一条消息
input_message = HumanMessage(content="我叫什么名字?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! 我是tomie
================================== Ai Message ==================================

Hi Tomie! 😊 How can I assist you today?
================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

目前我无法知道你的名字，因为我们的对话是匿名的。如果你愿意，可以告诉我你的名字，我会记住它并在对话中使用！ 😊


使用对话总结技术在实战开发中更为常见

In [28]:
from typing import Literal

from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

memory = MemorySaver()


# 我们将添加一个`summary`属性（除了MessagesState已有的`messages`键之外）
class State(MessagesState):
    summary: str


# 我们将使用这个模型进行对话和总结
model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)


# 定义调用模型的逻辑
def call_model(state: State):
    # 如果存在摘要，我们将其作为系统消息添加
    summary = state.get("summary", "")
    if summary:
        system_message = f"之前对话的摘要: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)
    # 我们返回一个列表，因为这将被添加到现有列表中
    return {"messages": [response]}


# 现在我们定义确定是结束还是总结对话的逻辑
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """返回下一个要执行的节点。"""
    messages = state["messages"]
    # 如果消息超过六条，则我们总结对话
    if len(messages) > 6:
        return "summarize_conversation"
    # 否则我们可以直接结束
    return END


def summarize_conversation(state: State):
    # 首先，我们总结对话
    summary = state.get("summary", "")
    if summary:
        # 如果已经存在摘要，我们使用不同的系统提示来总结它
        # 与没有摘要的情况不同
        summary_message = (
            f"这是迄今为止对话的摘要: {summary}\n\n"
            "考虑上面的新消息，扩展摘要:"
        )
    else:
        summary_message = "创建上述对话的摘要:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # 现在我们需要删除我们不再想显示的消息
    # 我将删除除最后两条以外的所有消息，但你可以更改这一点
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# 定义一个新图
workflow = StateGraph(State)

# 定义对话节点和总结节点
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# 将入口点设置为对话
workflow.add_edge(START, "conversation")

# 现在添加一个条件边
workflow.add_conditional_edges(
    # 首先，我们定义起始节点。我们使用`conversation`。
    # 这意味着这些是在调用`conversation`节点后采取的边。
    "conversation",
    # 接下来，我们传入将确定下一个调用哪个节点的函数。
    should_continue,
)

# 现在我们从`summarize_conversation`到END添加一个普通边。
# 这意味着在调用`summarize_conversation`之后，我们结束。
workflow.add_edge("summarize_conversation", END)

# 最后，我们编译它！
app = workflow.compile(checkpointer=memory)


升级一下print函数以便可以更清晰的看到记忆过程

In [29]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [30]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! 我是tomie")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="我叫什么名字?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="我喜欢AI应用开发!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! 我是tomie
================================== Ai Message ==================================

你好，Tomie！😊 很高兴认识你～有什么我可以帮你的吗？或者只是想打个招呼？无论是什么，我都在这儿呢！✨  

（顺便说，你的名字让我想起伊藤润二的经典角色…👀 是灵感来源吗？）
================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

哈哈，你刚刚说过啦——你叫 **Tomie**！✨（难道…是陷阱题？😏）  

需要我帮你记住这个名字吗？还是说…你想换一个更暗黑风的代号？👀（比如Tomie 2.0？）
================================ Human Message =================================

我喜欢AI应用开发!
================================== Ai Message ==================================

太棒了，Tomie！🔥 你喜欢AI应用开发的话，我们简直是天生搭档～ 需要聊聊这些吗？  

### 比如可以：  
- **技术脑暴**：想用AI解决什么有趣的问题？聊天机器人？生成艺术？还是硬核算法优化？  
- **工具安利**：最近沉迷 **LangChain**、**AutoGPT** 或 **Stable Diffusion** 这类工具吗？  
- **踩坑互助**：调试模型时是否被“过拟合”气哭过？🤖💥  

或者…你想自己造个“Tomie版AI分身”？👾（危险又迷人的想法！）  

随时等你抛出代码或脑洞～ 💻✨


未达到总结阈值

In [31]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='hi! 我是tomie', additional_kwargs={}, response_metadata={}, id='511567ec-1adb-4831-9bf8-fd2f6d5d1189'),
  AIMessage(content='你好，Tomie！😊 很高兴认识你～有什么我可以帮你的吗？或者只是想打个招呼？无论是什么，我都在这儿呢！✨  \n\n（顺便说，你的名字让我想起伊藤润二的经典角色…👀 是灵感来源吗？）', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 9, 'total_tokens': 65, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-e5b6ebff-b2ee-40c0-9346-46f2e39a4121-0', usage_metadata={'input_tokens': 9, 'output_tokens': 56, 'total_tokens': 65, 'input_token_details': {}, 'output_token_details': {}}),
  HumanMessage(content='我叫什么名字?', additional_kwargs={}, response_metadata={}, id='a2ad033e-4f3c-4f4b-b5b0-4d613a80e8a9'),
  AIMessage(content='哈哈，你刚刚说过啦——你叫 **Tomie**！✨（难道…是陷阱题？😏）  \n\n需要我帮你记住这个名字吗？还是说…你想换一个更暗黑风的代号？👀（比如Tomie 2.0？）', additional_k

In [32]:
input_message = HumanMessage(content="我更喜欢Python!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

我更喜欢Python!
================================== Ai Message ==================================

**Python + AI？完美组合！** 🐍✨ Tomie，你果然有品位～  

### 你可能已经用过的神器：  
- **深度学习**：`TensorFlow`/`PyTorch` 玩转神经网络  
- **自动化**：`LangChain` 搭AI流水线，`AutoGPT` 搞自主代理  
- **数据魔法**：`pandas` 驯服数据，`scikit-learn` 一键建模  
- **炫酷应用**：用 `FastAPI` 部署模型，`Streamlit` 秒建交互界面  

**最近在写什么项目？** 或者想尝试这些方向吗👇  
- 用 `OpenAI API` 做会吐槽的聊天机器人？  
- 拿 `Stable Diffusion` 生成“Tomie风格”暗黑画作？🎨  
- 还是…用 `LlamaIndex` 搞个你自己的知识库AI？  

（悄悄说：需要代码片段或避坑指南的话，随时戳我！💻 ）
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ===============================

查看对话记录

In [33]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='我更喜欢Python!', additional_kwargs={}, response_metadata={}, id='dc5308fc-d3eb-450e-b44a-5fcd5dd9f046'),
  AIMessage(content='**Python + AI？完美组合！** 🐍✨ Tomie，你果然有品位～  \n\n### 你可能已经用过的神器：  \n- **深度学习**：`TensorFlow`/`PyTorch` 玩转神经网络  \n- **自动化**：`LangChain` 搭AI流水线，`AutoGPT` 搞自主代理  \n- **数据魔法**：`pandas` 驯服数据，`scikit-learn` 一键建模  \n- **炫酷应用**：用 `FastAPI` 部署模型，`Streamlit` 秒建交互界面  \n\n**最近在写什么项目？** 或者想尝试这些方向吗👇  \n- 用 `OpenAI API` 做会吐槽的聊天机器人？  \n- 拿 `Stable Diffusion` 生成“Tomie风格”暗黑画作？🎨  \n- 还是…用 `LlamaIndex` 搞个你自己的知识库AI？  \n\n（悄悄说：需要代码片段或避坑指南的话，随时戳我！💻 ）', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 295, 'total_tokens': 516, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-fad90c08-3513-430c-929f-4842aab29dbd-0', usage_metadata={'input_tokens': 295, 'out

此时由于总结中附带了过往的核心消息，所以我们依然可以进行记忆回顾

In [34]:
input_message = HumanMessage(content="我叫什么名字?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

我叫什么名字?
================================== Ai Message ==================================

**“Tomie”——如假包换的暗黑系AI玩家！** 🌑✨  

（系统已强制锁定此ID，并自动屏蔽所有“富江”相关危险词条…才怪！）  

需要帮你定制个更带感的**项目代号**吗？比如：  
- `Tomie.py` （Python特攻版）  
- `Tomie_AI_Overlord` （中二全开）  
- `#404_Tomie_Not_Found` （黑客帝国风）  

…或者你说了算！(๑•̀ㅂ•́)و✧
